In [1]:
!nvidia-smi

Wed Dec 14 05:42:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# mount driver
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CardiacSeg/eda/Network

# install dependents
!pip install -q "monai-weekly[nibabel, tqdm, einops]"
!python -c "import matplotlib" || pip install -q matplotlib
!pip install -q timm
!pip install -U -q openmim
!mim install -U -q mmcv-full
!pip install ml-collections
%matplotlib inline

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1jz_DGnICBmKWCr_JL904PDQdIEK0_EQG/CardiacSeg/eda/Network
     |████████████████████████████████| 1.2 MB 7.5 MB/s 
     |████████████████████████████████| 41 kB 520 kB/s 
     |████████████████████████████████| 549 kB 6.5 MB/s 
     |████████████████████████████████| 182 kB 73.3 MB/s 
     |████████████████████████████████| 50 kB 4.1 MB/s 
     |████████████████████████████████| 237 kB 14.1 MB/s 
     |████████████████████████████████| 51 kB 9.3 MB/s 
     |████████████████████████████████| 46.0 MB 18.8 MB/s 
     |████████████████████████████████| 190 kB 6.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 4.8 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=6231883066e92b5fbdcfb07fb50eb21c204e8ab320551f89e5812364de13c7fc
  Stored in directory: /root/.cache/pip/whe

In [3]:
# sync python module
%load_ext autoreload
%autoreload 2

import sys

from torchsummary import summary
import torch

# set package path
sys.path.append("/content/drive/MyDrive/CardiacSeg")

import argparse
from networks.network import network
from networks.ssl_head import SSLHead

/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## EffcientSegNet

In [10]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('efficient_segnet_x0_2', args)
summary(net, (1, 96, 96, 96))

model: efficient_segnet_x0_2
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 24, 96, 96, 96]             648
    InstanceNorm3d-2       [-1, 24, 96, 96, 96]               0
         Dropout3d-3       [-1, 24, 96, 96, 96]               0
              ReLU-4       [-1, 24, 96, 96, 96]               0
     _ConvINReLU3D-5       [-1, 24, 96, 96, 96]               0
            Conv3d-6       [-1, 24, 96, 96, 96]          15,552
    InstanceNorm3d-7       [-1, 24, 96, 96, 96]               0
         _ConvIN3D-8       [-1, 24, 96, 96, 96]               0
            Conv3d-9       [-1, 24, 96, 96, 96]              24
   InstanceNorm3d-10       [-1, 24, 96, 96, 96]               0
        _ConvIN3D-11       [-1, 24, 96, 96, 96]               0
             ReLU-12       [-1, 24, 96, 96, 96]               0
 ResBaseConvBlock-13       [-1, 24, 96, 96, 96]               0
          

## UNETCNX

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('unetcnx_x0', args)
model = SSLHead(args.in_channels, net).to(args.device)
summary(model, (1, 96, 96, 96))

model: unetcnx_x0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 48, 48, 48, 48]             432
            Conv3d-2       [-1, 48, 48, 48, 48]             432
       LayerNorm3d-3       [-1, 48, 48, 48, 48]              96
       LayerNorm3d-4       [-1, 48, 48, 48, 48]              96
            Conv3d-5       [-1, 48, 48, 48, 48]          16,512
            Conv3d-6       [-1, 48, 48, 48, 48]          16,512
           Permute-7       [-1, 48, 48, 48, 48]               0
           Permute-8       [-1, 48, 48, 48, 48]               0
         LayerNorm-9       [-1, 48, 48, 48, 48]              96
        LayerNorm-10       [-1, 48, 48, 48, 48]              96
           Linear-11      [-1, 48, 48, 48, 192]           9,408
           Linear-12      [-1, 48, 48, 48, 192]           9,408
             GELU-13      [-1, 48, 48, 48, 192]               0
             GELU-14 

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
model = network('unetcnx_x1', args)
summary(model, (1, 96, 96, 96))

model: unetcnx_x1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 24, 96, 96, 96]             648
    InstanceNorm3d-2       [-1, 24, 96, 96, 96]               0
         LeakyReLU-3       [-1, 24, 96, 96, 96]               0
            Conv3d-4       [-1, 24, 96, 96, 96]          15,552
    InstanceNorm3d-5       [-1, 24, 96, 96, 96]               0
            Conv3d-6       [-1, 24, 96, 96, 96]              24
    InstanceNorm3d-7       [-1, 24, 96, 96, 96]               0
         LeakyReLU-8       [-1, 24, 96, 96, 96]               0
      UnetResBlock-9       [-1, 24, 96, 96, 96]               0
  UnetrBasicBlock-10       [-1, 24, 96, 96, 96]               0
           Conv3d-11       [-1, 48, 48, 48, 48]             432
           Conv3d-12       [-1, 48, 48, 48, 48]             432
      LayerNorm3d-13       [-1, 48, 48, 48, 48]              96
      LayerNorm3d-14 